In [184]:
from glob import glob
import pandas as pd
from collections import defaultdict

In [185]:
datasets = ["gsm8k", "rte", "sst-5", "agnews", "subj"]  # "",
optimizers = ["EvoPromptGA", "CAPO"]
models = ["llama", "mistral", "qwen"]
seeds = [42, 43, 44]

In [186]:
dfs = defaultdict(pd.DataFrame)

for d in datasets:
    for o in optimizers:
        for m in models:
            for s in seeds:
                path = f"../results/{d}/{m}/{o}/seed{s}/*/*/step_results_eval.csv"
                files = glob(path)
                if len(files) != 1:
                    print(f"weird number of files: {len(files)} for {path}")
                    continue
                file = files[0]
                df = pd.read_csv(file)
                tokens_per_step = (
                    df.groupby("step").first()["input_tokens_meta_llm"]
                    + df.groupby("step").first()["input_tokens_downstream_llm"]
                )
                # map back to steps
                df["cum_token"] = df["step"].map(tokens_per_step.cumsum())
                dfs[(d, o, m, s)] = df

weird number of files: 0 for ../results/subj/mistral/CAPO/seed43/*/*/step_results_eval.csv
weird number of files: 0 for ../results/subj/mistral/CAPO/seed44/*/*/step_results_eval.csv


In [187]:
CUTOFF_TOKENS = 5_000_000

In [199]:
model = "mistral"
results = {"optimizer": [], "dataset": [], "mean": [], "std": []}
# calculate mean and std per dataset and optimizer at CUTOFF_STEP
for d in datasets:
    for o in optimizers:
        # find first step that is under CUTOFF for each dataframe
        steps = []
        for s in seeds:
            try:
                df = dfs[(d, o, model, s)].assign(seed=s)
            except KeyError:
                print(f"missing {d} {o} {model} {s}")
                continue
            if len(df) == 0:
                continue
            last_step = df.loc[df["cum_token"] < CUTOFF_TOKENS, "step"].max()
            # take last step with max test_score
            df = df[df["step"] == last_step].nlargest(1, "test_score")
            steps.append(df)
        df = pd.concat(steps)

        results["mean"].append(df["test_score"].mean())
        results["std"].append(df["test_score"].std())

        results["optimizer"].append(o)
        results["dataset"].append(d)

In [200]:
df = pd.DataFrame(results)
df = df.set_index("optimizer")
df = df.pivot(columns="dataset")
df["avg"] = df["mean"].mean(axis=1).mul(100).round(2)
df["mean"] = df["mean"].mul(100).round(1)
df["std"] = df["std"].mul(100).round(1)

In [201]:
df["mean"] = (
    df["mean"].astype(str).apply(lambda x: x[:5])
    + "±"
    + df["std"].astype(str).apply(lambda x: x[:5])
)
df = df.drop(columns=["std"])
print(model)
df

mistral


mean                                            avg
dataset        agnews     gsm8k       rte     sst-5      subj       
optimizer                                                           
CAPO         89.0±2.0  62.5±1.1  90.0±0.2  60.9±0.9  86.6±nan  77.80
EvoPromptGA  84.7±1.7  45.9±5.5  88.0±1.5  53.1±1.3  75.4±2.7  69.41

In [202]:
df.reset_index().droplevel(0, axis=1)  # .set_index("")

dataset,,agnews,gsm8k,rte,sst-5,subj,
0,CAPO,89.0±2.0,62.5±1.1,90.0±0.2,60.9±0.9,86.6±nan,77.80
1,EvoPromptGA,84.7±1.7,45.9±5.5,88.0±1.5,53.1±1.3,75.4±2.7,69.41
